import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.utils import to_categorical

In [1]:
import pandas

In [2]:
df = pandas.read_csv("C:\\Users\\sante\\Desktop\\мой нлп - Лист1-2.csv", encoding="utf-8")

# тестовые данные
valid = df["title"].isin(['Интерстеллар', "Омерзительная восьмерка", "Тройной форсаж: Токийский дрифт"])
test = df[valid]
del test['title']

# удалили наши данные(оставили данные для обучения)
df = df.loc[df['title'] != "Интерстеллар"]
df = df.loc[df['title'] != "Омерзительная восьмерка"]
df = df.loc[df['title'] != "Тройной форсаж: Токийский дрифт"]
del df['title']

In [3]:
 df.head

<bound method NDFrame.head of                                                   text  label
0    Дон Блут – один из самых известных анимационны...     -1
1    Отрицательных рецензий на этот мультфильм очен...     -1
2    \nЧестно признаюсь: в детстве я если и смотрел...     -1
3    \nВпервые я познакомилась с творением студии '...     -1
4    \nЯ узнала про мультфильм 'Анастасия' на продл...     -1
..                                                 ...    ...
585  Питер Джексон уже не тот...  Пишу как человек,...     -1
586  Ждали-ждали - дождались  Около двух лет назад ...     -1
587  Три часа скукоты  В том, что «Хоббит: Нежданно...     -1
588  что-то случилось с любимым нами Средиземьем  В...     -1
589  Детям фильм смотреть нельзя  Фильм не понравил...     -1

[590 rows x 2 columns]>

In [4]:
def tokenizeWord(word):
    word = tokenizer.tokenize(word)
    if len(word) == 1:
        word = word[0]
    else:
        word = ""
    return word


In [5]:

import pymorphy2
from nltk.tokenize import RegexpTokenizer
morph = pymorphy2.MorphAnalyzer()
tokenizer = RegexpTokenizer(r'\w+')
uniqWords = []
tupleIndex = 0
sentences = []
for tuple in df.values:
    tupleIndex+=1
    words = tuple[0].split()
    wordsList = []
    for word in words:
        wordNF = tokenizeWord(morph.parse(word)[0].normal_form)
        uniqWords.append(wordNF)
        wordsList.append(wordNF)
    sentences.append(wordsList)
tupleIndex = 0
testSentences = []
for tuple in test.values:
    tupleIndex+=1
    words = tuple[0].split()
    wordsList = []
    for word in words:
        wordNF = tokenizeWord(morph.parse(word)[0].normal_form)
        uniqWords.append(wordNF)
        wordsList.append(wordNF)
    testSentences.append(wordsList)

In [6]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

print(len(uniqWords))

tokenizer = Tokenizer(num_words=len(uniqWords))
tokenizer.fit_on_texts(sentences)

vocab_size = len(tokenizer.word_index) + 1
maxlen = 1000

X_train = tokenizer.texts_to_sequences(sentences)
X_test = tokenizer.texts_to_sequences(testSentences)

Using TensorFlow backend.


273477


In [7]:
X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

In [8]:
print(vocab_size)


25916


In [9]:
from keras.utils import to_categorical
y_train = df["label"]
y_test = test["label"]

num_classes = 3

y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)


In [34]:
import numpy as np
def create_embedding_matrix(filepath, word_index, embedding_dim):
    vocab_size = len(word_index) + 1  # Adding again 1 because of reserved 0 index
    embedding_matrix = np.zeros((vocab_size, embedding_dim))

    with open(filepath, encoding="utf-8") as f:
        for line in f:
            word, *vector = line.split()
            word = word.split("_")[0]
            if word in word_index:
                idx = word_index[word] 
                embedding_matrix[idx] = np.array(
                    vector, dtype=np.float32)[:embedding_dim]

    return embedding_matrix

In [42]:
embedding_dim = 300
embedding_matrix = create_embedding_matrix('C:/Users/sante/Desktop/model.txt', tokenizer.word_index, embedding_dim)

In [43]:
nonzero_elements = np.count_nonzero(np.count_nonzero(embedding_matrix, axis=1))
print(embedding_matrix.shape)
print(nonzero_elements)
nonzero_elements / vocab_size

(25916, 300)
14052


0.5422133045223029

In [44]:
from keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D
from keras.models import Sequential
from keras import layers
model = Sequential()
model.add(layers.Embedding(vocab_size, embedding_dim, 
                           weights=[embedding_matrix], 
                           input_length=maxlen, 
                           trainable=True))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(num_classes))
model.add(Activation('softmax'))
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 1000, 300)         7774800   
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 998, 128)          115328    
_________________________________________________________________
activation_5 (Activation)    (None, 998, 128)          0         
_________________________________________________________________
global_max_pooling1d_3 (Glob (None, 128)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 3)                 387       
_________________________________________________________________
activation_6 (Activation)    (None, 3)                 0         
Total params: 7,890,515
Trainable params: 7,890,515
Non-trainable params: 0
____________________________________________

In [45]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


In [46]:
from sklearn.metrics import classification_report
model.fit(X_train, y_train, epochs=10, batch_size=128)
predictions = model.predict(X_test)
print(classification_report(y_test.argmax(axis=1), predictions.argmax(axis=1)))

C:\Users\sante\anaconda3\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/10
590/590 [==============================] - 6s 10ms/step - loss: 2.4841 - accuracy: 0.3102
Epoch 2/10
590/590 [==============================] - 6s 10ms/step - loss: 1.4417 - accuracy: 0.4593
Epoch 3/10
590/590 [==============================] - 6s 10ms/step - loss: 0.6573 - accuracy: 0.7475
Epoch 4/10
590/590 [==============================] - 6s 10ms/step - loss: 0.4649 - accuracy: 0.8356
Epoch 5/10
590/590 [==============================] - 6s 10ms/step - loss: 0.2536 - accuracy: 0.9508
Epoch 6/10
590/590 [==============================] - 6s 10ms/step - loss: 0.1793 - accuracy: 0.9847
Epoch 7/10
590/590 [==============================] - 6s 10ms/step - loss: 0.1112 - accuracy: 0.9983
Epoch 8/10
590/590 [==============================] - 6s 10ms/step - loss: 0.0980 - accuracy: 0.9983
Epoch 9/10
590/590 [==============================] - 6s 10ms/step - loss: 0.0668 - accuracy: 0.9983
Epoch 10/10
590/590 [==============================] - 6s 10ms/step - loss: 0.0579 - accura

In [47]:
model.fit(X_train, y_train, epochs=20, batch_size=128)
predictions = model.predict(X_test)
print(classification_report(y_test.argmax(axis=1), predictions.argmax(axis=1)))

Epoch 1/20
590/590 [==============================] - 6s 10ms/step - loss: 0.0503 - accuracy: 0.9983
Epoch 2/20
590/590 [==============================] - 6s 9ms/step - loss: 0.0425 - accuracy: 0.9983
Epoch 3/20
590/590 [==============================] - 6s 10ms/step - loss: 0.0393 - accuracy: 0.9983
Epoch 4/20
590/590 [==============================] - 6s 9ms/step - loss: 0.0356 - accuracy: 0.9983
Epoch 5/20
590/590 [==============================] - 6s 10ms/step - loss: 0.0324 - accuracy: 0.9983
Epoch 6/20
590/590 [==============================] - 6s 10ms/step - loss: 0.0302 - accuracy: 0.9983
Epoch 7/20
590/590 [==============================] - 6s 10ms/step - loss: 0.0274 - accuracy: 0.9983
Epoch 8/20
590/590 [==============================] - 6s 10ms/step - loss: 0.0261 - accuracy: 0.9983
Epoch 9/20
590/590 [==============================] - 6s 10ms/step - loss: 0.0251 - accuracy: 0.9983
Epoch 10/20
590/590 [==============================] - 6s 9ms/step - loss: 0.0243 - accuracy:

In [48]:
model.fit(X_train, y_train, epochs=30, batch_size=128)
predictions = model.predict(X_test)
print(classification_report(y_test.argmax(axis=1), predictions.argmax(axis=1)))

Epoch 1/30
590/590 [==============================] - 6s 10ms/step - loss: 0.0159 - accuracy: 0.9983
Epoch 2/30
590/590 [==============================] - 6s 10ms/step - loss: 0.0184 - accuracy: 0.9966
Epoch 3/30
590/590 [==============================] - 6s 10ms/step - loss: 0.0166 - accuracy: 0.9983
Epoch 4/30
590/590 [==============================] - 6s 10ms/step - loss: 0.0165 - accuracy: 0.9983
Epoch 5/30
590/590 [==============================] - 6s 10ms/step - loss: 0.0156 - accuracy: 0.9983
Epoch 6/30
590/590 [==============================] - 6s 10ms/step - loss: 0.0136 - accuracy: 0.9983
Epoch 7/30
590/590 [==============================] - 6s 10ms/step - loss: 0.0127 - accuracy: 0.9983
Epoch 8/30
590/590 [==============================] - 6s 10ms/step - loss: 0.0130 - accuracy: 0.9983
Epoch 9/30
590/590 [==============================] - 6s 9ms/step - loss: 0.0139 - accuracy: 0.9983
Epoch 10/30
590/590 [==============================] - 6s 10ms/step - loss: 0.0138 - accurac